<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Karpov_ML/blob/main/Stepik_group_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#edu1

In [69]:
import pandas as pd
import numpy as np

In [70]:
# student_performance = pd.read_csv('https://stepik.org/media/attachments/course/4852/StudentsPerformance.csv')
students_performance = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/Stepik_Karpov_ML/main/StudentsPerformance.csv')
students_performance.columns = [x.replace(" ", "_") for x in students_performance.columns]

In [71]:
students_performance.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Подсчитаем среднее значение скоров, в зависимости от гендера.

In [72]:
students_performance.groupby('gender')

Первое - это уже валидное высказывание. Результат - сгруппированный пандасовский датафрейм.

In [73]:
students_performance.groupby('gender').head(2)

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Теперь к нему можно применить аггрегацию. Самый простой способ - просто написать функцию, которую применияем. Получим среднее значение наших количественных переменных, для группирующей переменной.

In [74]:
students_performance.groupby('gender').mean()

,math_score,reading_score,writing_score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


In [75]:
students_performance.groupby('race/ethnicity').mean()

,math_score,reading_score,writing_score
race/ethnicity,,,
group A,61.629213,64.674157,62.674157
group B,63.452632,67.352632,65.600000
group C,64.463950,69.103448,67.827586
group D,67.362595,70.030534,70.145038
group E,73.821429,73.028571,71.407143


Но если хотели бы несколько функций, то это было бы уже затруднительно. Можем более подробно расписать как именно мы хотим сделать группировку. Воспользуемся методом аггрегейт.

В агг можно передать словарь с маппингом того, с какие переменные мы хотим использовать для аггрегации и что мы про эти переменные хотим узнать.

Чтобы увидеть надпись лейбл чего и название метрики

In [76]:
students_performance.groupby('gender').aggregate({"math_score":'mean'})

,math_score
gender,
female,63.633205
male,68.728216


In [77]:
students_performance.groupby('gender').aggregate({"math_score":'mean', "reading_score": 'mean'})

,math_score,reading_score
gender,,
female,63.633205,72.608108
male,68.728216,65.473029


Результатом аггреации будет пандасовский датафрейм, где индексом будет значение нашей переменной, по которой мы проводили аггрегацию.

Не всегда мы хотим, чтобы индексом были значения нашей переменной, и тогда можно поставить эз индекс = фалс.

In [78]:
students_performance.groupby('gender', as_index=False).aggregate({"math_score":'mean', "reading_score": 'mean'})

,gender,math_score,reading_score
0,female,63.633205,72.608108
1,male,68.728216,65.473029


Правда не совсем удобно, что в самом названии колонок никак не заложено, что это значение средних.

Можно сделать ренейм и переименновать интересующие нас колонки.

In [79]:
students_performance.groupby('gender').\
agg({'math_score': 'mean', "reading_score": 'mean'}).\
rename(columns={'math_score': 'math_score_mean', 'reading_score': 'reading_score_mean'})

,math_score_mean,reading_score_mean
gender,,
female,63.633205,72.608108
male,68.728216,65.473029


In [80]:
students_performance.groupby('gender').agg({'math_score': ['mean'], "reading_score": ['mean']})

,math_score,reading_score
,mean,mean
gender,,
female,63.633205,72.608108
male,68.728216,65.473029


In [81]:
students_performance.groupby('gender', as_index=False)\
  .aggregate({'math_score': ['mean', 'count', 'std'],
              'reading_score': ['std', 'min', 'max']})

gender math_score                  reading_score         
                mean count        std           std min  max
0  female  63.633205   518  15.491453     14.378245  17  100
1    male  68.728216   482  14.356277     13.931832  23  100

Нам также ничего не мешает группировать наши наблюдения сразу по нескольким переменным. 

In [82]:
students_performance.groupby(['gender', 'race/ethnicity'], as_index=False).\
agg({'math_score': 'mean', "reading_score": 'mean'}).\
rename(columns={'math_score': 'math_score_mean', 'reading_score': 'reading_score_mean'}).\
head(10)

,gender,race/ethnicity,math_score_mean,reading_score_mean
0,female,group A,58.527778,69.000000
1,female,group B,61.403846,71.076923
2,female,group C,62.033333,71.944444
3,female,group D,65.248062,74.046512
4,female,group E,70.811594,75.840580
5,male,group A,63.735849,61.735849
6,male,group B,65.930233,62.848837
7,male,group C,67.611511,65.424460
8,male,group D,69.413534,66.135338
9,male,group E,76.746479,70.295775


Если убрать индексы, то получим сложный составной индекс.

In [83]:
students_performance.groupby(['gender', 'race/ethnicity']).\
agg({'math_score': 'mean', "reading_score": 'mean'}).\
rename(columns={'math_score': 'math_score_mean', 'reading_score': 'reading_score_mean'}).\
head(10)

math_score_mean  reading_score_mean
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923
       group C               62.033333           71.944444
       group D               65.248062           74.046512
       group E               70.811594           75.840580
male   group A               63.735849           61.735849
       group B               65.930233           62.848837
       group C               67.611511           65.424460
       group D               69.413534           66.135338
       group E               76.746479           70.295775

Т.е. возможна и такая сложная штука. Можем сохранить наш датафрейм в другой.

In [84]:
mean_scores = students_performance.groupby(['gender', 'race/ethnicity']).\
agg({'math_score': 'mean', "reading_score": 'mean'}).\
rename(columns={'math_score': 'math_score_mean', 'reading_score': 'reading_score_mean'})

mean_scores

math_score_mean  reading_score_mean
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923
       group C               62.033333           71.944444
       group D               65.248062           74.046512
       group E               70.811594           75.840580
male   group A               63.735849           61.735849
       group B               65.930233           62.848837
       group C               67.611511           65.424460
       group D               69.413534           66.135338
       group E               76.746479           70.295775

In [85]:
mean_scores.index

MultiIndex([('female', 'group A'),
            ('female', 'group B'),
            ('female', 'group C'),
            ('female', 'group D'),
            ('female', 'group E'),
            (  'male', 'group A'),
            (  'male', 'group B'),
            (  'male', 'group C'),
            (  'male', 'group D'),
            (  'male', 'group E')],
           names=['gender', 'race/ethnicity'])

Т.е. у нас появляются мультииндексы, состаящие из нескольких индексов, где более глобальным будет пол - гендер, а внутри подгруппировка по значениям переменной рейс. Но вообще такие индексы усложняют работу с датафреймом. и лучше вернуться и установить эз_индекс = Фалс. Но иногда можно использовать и их. 

Всё также мы можем использовать лейблы для отбора нужных нам строчек.

In [86]:
mean_scores.loc[('female', 'group A')]

math_score_mean       58.527778
reading_score_mean    69.000000
Name: (female, group A), dtype: float64

In [87]:
mean_scores.loc[[('female', 'group A'), ('female', 'group B')]]

math_score_mean  reading_score_mean
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923

Но есть преимущества и у мультииндексов. Мы знаем, что наши колонки - это пандасовские серии.

In [88]:
students_performance.math_score.head(5)

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

Можно вывести все уникальные значения.

In [89]:
students_performance.math_score.unique()

array([ 72,  69,  90,  47,  76,  71,  88,  40,  64,  38,  58,  65,  78,
        50,  18,  46,  54,  66,  44,  74,  73,  67,  70,  62,  63,  56,
        97,  81,  75,  57,  55,  53,  59,  82,  77,  33,  52,   0,  79,
        39,  45,  60,  61,  41,  49,  30,  80,  42,  27,  43,  68,  85,
        98,  87,  51,  99,  84,  91,  83,  89,  22, 100,  96,  94,  48,
        35,  34,  86,  92,  37,  28,  24,  26,  95,  36,  29,  32,  93,
        19,  23,   8])

Или посчитать число уникальных значений.

In [90]:
students_performance.math_score.nunique()

81

Если бы мы хотели посмотреть сколько у нас уникальных скоров по математике на пересечении двух групп.

In [44]:
students_performance.groupby(['gender', 'race/ethnicity']).math_score

Это уже сгруппированная серия. Т.е. это всё ещё одномерный массив, однако, мы к нему добавили информацию о группировке сразу по двум переменным.

In [91]:
students_performance.groupby(['gender', 'race/ethnicity']).math_score.nunique()

gender  race/ethnicity
female  group A           29
        group B           51
        group C           59
        group D           53
        group E           44
male    group A           38
        group B           43
        group C           56
        group D           49
        group E           38
Name: math_score, dtype: int64

In [92]:
type(students_performance.groupby(['gender', 'race/ethnicity']).math_score.nunique())

pandas.core.series.Series

Groupby можно комбинировать не только с аггрегацией, но и с другими методами. Решим задачку: отберём по 5 топ студентов по математике внутри каждого значения по переменной пол. Т.е. топ 5 математиков - девушек и юношей. Сначала сортируем наши данные.

In [93]:
students_performance.sort_values(['gender', 'math_score'])

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
59,female,group C,some high school,free/reduced,none,0,17,10
980,female,group B,high school,free/reduced,none,8,24,23
17,female,group B,some high school,free/reduced,none,18,32,28
787,female,group B,some college,standard,none,19,38,32
145,female,group C,some college,free/reduced,none,22,39,33
...,...,...,...,...,...,...,...,...
306,male,group E,some college,standard,completed,99,87,81
149,male,group E,associate's degree,free/reduced,completed,100,100,93
623,male,group A,some college,standard,completed,100,96,86
625,male,group D,some college,standard,completed,100,97,99


Т.е. тут уже получили отсортированных по математике внутри нашей переменной пол. В данном случае, отсортированы от наименьшего к наибольшему.

In [94]:
students_performance.sort_values(['gender', 'math_score'], ascending=False)

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
149,male,group E,associate's degree,free/reduced,completed,100,100,93
623,male,group A,some college,standard,completed,100,96,86
625,male,group D,some college,standard,completed,100,97,99
916,male,group E,bachelor's degree,standard,completed,100,100,100
306,male,group E,some college,standard,completed,99,87,81
...,...,...,...,...,...,...,...,...
145,female,group C,some college,free/reduced,none,22,39,33
787,female,group B,some college,standard,none,19,38,32
17,female,group B,some high school,free/reduced,none,18,32,28
980,female,group B,high school,free/reduced,none,8,24,23


In [95]:
students_performance.sort_values(['gender', 'math_score'], ascending=False).\
groupby('gender').head(5)

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
149,male,group E,associate's degree,free/reduced,completed,100,100,93
623,male,group A,some college,standard,completed,100,96,86
625,male,group D,some college,standard,completed,100,97,99
916,male,group E,bachelor's degree,standard,completed,100,100,100
306,male,group E,some college,standard,completed,99,87,81
451,female,group E,some college,standard,none,100,92,97
458,female,group E,bachelor's degree,standard,none,100,100,100
962,female,group E,associate's degree,standard,none,100,100,100
114,female,group E,bachelor's degree,standard,completed,99,100,100
263,female,group E,high school,standard,none,99,93,90


##Отступление.

Хочу попробовать сгруппировать по race. Но по определённому значению, по группе B.

In [96]:
students_performance.groupby(students_performance['race/ethnicity'] == 'group B').mean()

,math_score,reading_score,writing_score
race/ethnicity,,,
False,66.707407,69.595062,68.62963
True,63.452632,67.352632,65.60000


In [97]:
students_performance[students_performance['race/ethnicity'] == 'group B'].head(10)

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
2,female,group B,master's degree,standard,none,90,95,93
5,female,group B,associate's degree,standard,none,71,83,78
6,female,group B,some college,standard,completed,88,95,92
7,male,group B,some college,free/reduced,none,40,43,39
9,female,group B,high school,free/reduced,none,38,60,50
12,female,group B,high school,standard,none,65,81,73
17,female,group B,some high school,free/reduced,none,18,32,28
21,female,group B,some college,free/reduced,completed,65,75,70
26,male,group B,some college,standard,none,69,54,55


Всё таки сначала нужно отфильтровать по группе, а потом группировать.

In [98]:
students_performance[students_performance['race/ethnicity'] == 'group B'].groupby('race/ethnicity').mean()

,math_score,reading_score,writing_score
race/ethnicity,,,
group B,63.452632,67.352632,65.6


#edu2

Как создавать новые колонки в датафреймах. Добавление данных. Датафрейм - это словарь, который по именам хранит в себе серии. Так что просто заведём новый элемент.

Пусть хотим добавить тотальную успеваемость наших студентов. Серии можно складывать друг с другом. Это прямолинейный способ

In [99]:
students_performance.math_score + students_performance.reading_score + students_performance.writing_score

0      218
1      247
2      278
3      148
4      229
      ... 
995    282
996    172
997    195
998    223
999    249
Length: 1000, dtype: int64

Также просуммировать можно по-другому. Отфильтровали по скору колонки и сложили

In [100]:
students_performance.filter(like = 'score').sum(axis=1)

0      218
1      247
2      278
3      148
4      229
      ... 
995    282
996    172
997    195
998    223
999    249
Length: 1000, dtype: int64

In [101]:
students_performance['total_score'] = students_performance.math_score + students_performance.reading_score + students_performance.writing_score
students_performance.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


Это простое добавление одной колонки. Но нужно сделать множественное преобразование(например, одну колонку прологарифмировать, другую - сложить с чем-нибудь, третью - вычесть и т.д.) - для таких случаев используется метод assign. Кроме того assign создаёт копию датафрейма.

Заведём тотал_скор_лог и посчитаем логарифм нашей колони тотал_скор. Опять же тотал_скор - это просто серия и её можно запихать в нампай лог.

In [107]:
students_performance = students_performance.assign(total_score_log = np.log(students_performance.total_score))
students_performance

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,total_score_log
0,female,group B,bachelor's degree,standard,none,72,72,74,218,5.384495
1,female,group C,some college,standard,completed,69,90,88,247,5.509388
2,female,group B,master's degree,standard,none,90,95,93,278,5.627621
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,4.997212
4,male,group C,some college,standard,none,76,78,75,229,5.433722
...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,282,5.641907
996,male,group C,high school,free/reduced,none,62,55,55,172,5.147494
997,female,group C,high school,free/reduced,completed,59,71,65,195,5.273000
998,female,group D,some college,standard,completed,68,78,77,223,5.407172


Научимся удалять колонки.

In [108]:
students_performance.head(5)

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,total_score_log
0,female,group B,bachelor's degree,standard,none,72,72,74,218,5.384495
1,female,group C,some college,standard,completed,69,90,88,247,5.509388
2,female,group B,master's degree,standard,none,90,95,93,278,5.627621
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,4.997212
4,male,group C,some college,standard,none,76,78,75,229,5.433722


In [109]:
students_performance.drop(['total_score'], axis=1).head(5)

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score_log
0,female,group B,bachelor's degree,standard,none,72,72,74,5.384495
1,female,group C,some college,standard,completed,69,90,88,5.509388
2,female,group B,master's degree,standard,none,90,95,93,5.627621
3,male,group A,associate's degree,free/reduced,none,47,57,44,4.997212
4,male,group C,some college,standard,none,76,78,75,5.433722


assign можно в скобках указывать множество новых полей

In [110]:
students_performance.drop(['total_score', 'lunch'], axis=1).head(5)

,gender,race/ethnicity,parental_level_of_education,test_preparation_course,math_score,reading_score,writing_score,total_score_log
0,female,group B,bachelor's degree,none,72,72,74,5.384495
1,female,group C,some college,completed,69,90,88,5.509388
2,female,group B,master's degree,none,90,95,93,5.627621
3,male,group A,associate's degree,none,47,57,44,4.997212
4,male,group C,some college,none,76,78,75,5.433722


drop не изменяет исходный датафрейм, то есть чтобы получить новый датафрейм без колонки lunch нам нужно или явно присвоить результат новому датафрейму:

In [111]:
students_performance.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,total_score_log
0,female,group B,bachelor's degree,standard,none,72,72,74,218,5.384495
1,female,group C,some college,standard,completed,69,90,88,247,5.509388
2,female,group B,master's degree,standard,none,90,95,93,278,5.627621
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,4.997212
4,male,group C,some college,standard,none,76,78,75,229,5.433722


#test1

#1

Пересчитаем число ног у героев игры Dota2! Сгруппируйте героев из датасэта по числу их ног (колонка legs), и заполните их число в задании ниже.

Данные взяты отсюда(https://api.opendota.com/api/heroes), на этом же сайте(https://www.opendota.com) можно найти больше разнообразных данных по Dota2.

In [112]:
#df = pd.read_csv('https://stepik.org/media/attachments/course/4852/dota_hero_stats.csv')
df = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/Stepik_Karpov_ML/main/dota_hero_stats.csv')
df.head()

,Unnamed: 0,attack_type,id,legs,localized_name,name,primary_attr,roles
0,0,Melee,1,2,Anti-Mage,npc_dota_hero_antimage,agi,"['Carry', 'Escape', 'Nuker']"
1,1,Melee,2,2,Axe,npc_dota_hero_axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']"
2,2,Ranged,3,4,Bane,npc_dota_hero_bane,int,"['Support', 'Disabler', 'Nuker', 'Durable']"
3,3,Melee,4,2,Bloodseeker,npc_dota_hero_bloodseeker,agi,"['Carry', 'Disabler', 'Jungler', 'Nuker', 'Ini..."
4,4,Ranged,5,2,Crystal Maiden,npc_dota_hero_crystal_maiden,int,"['Support', 'Disabler', 'Nuker', 'Jungler']"


In [ ]:
# df.count(axis=1).head(10)

In [118]:
#df.groupby('legs').agg('count')

In [114]:
df.groupby('legs').agg({'legs': ['count']})

,legs
,count
legs,
0,11
2,95
4,7
6,3
8,1


In [115]:
df.groupby('legs').size()

legs
0    11
2    95
4     7
6     3
8     1
dtype: int64

In [116]:
df.aggregate({'legs': 'value_counts'}).sort_index()

,legs
0,11
2,95
4,7
6,3
8,1


In [117]:
df.legs.value_counts().sort_index()

0    11
2    95
4     7
6     3
8     1
Name: legs, dtype: int64

Число героев с "0 ног" - 11

Число героев с 2-мя ногами - 95

Число героев с 4-мя ногами - 7

Число героев с 6-ю ногами - 3

Число героев с 8-ю ногами - 1


In [ ]:
from pandas.core.algorithms import value_counts
? value_counts

#2

К нам поступили данные из бухгалтерии о заработках(accountancy.csv) Лупы и Пупы за разные задачи! Посмотрите у кого из них больше средний заработок в различных категориях (колонка Type) и заполните таблицу, указывая исполнителя с большим заработком в каждой из категорий.

In [119]:
#df = pd.read_csv('https://stepik.org/media/attachments/course/4852/accountancy.csv')
df = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/Stepik_Karpov_ML/main/accountancy.csv')
df.head()

,Unnamed: 0,Executor,Type,Salary
0,0,Pupa,D,63
1,1,Pupa,A,158
2,2,Pupa,D,194
3,3,Pupa,E,109
4,4,Loopa,E,184


In [120]:
df.groupby(['Type', 'Executor']).agg({'Salary': ['mean']}).unstack()

Salary            
                mean            
Executor       Loopa        Pupa
Type                            
A          58.000000  160.833333
B         145.166667   77.000000
C         154.333333   74.500000
D         137.714286  146.500000
E         164.000000  131.200000
F         238.000000  136.250000

In [125]:
df.groupby(['Executor', 'Type']).agg({'Salary': 'mean'}).rename(columns={'Salary': 'Salary_mean'}).unstack()

Salary_mean                                                   
Type               A           B           C           D      E       F
Executor                                                               
Loopa      58.000000  145.166667  154.333333  137.714286  164.0  238.00
Pupa      160.833333   77.000000   74.500000  146.500000  131.2  136.25


Rows:	Loopa	Pupa

A	          pupa

B	  loopa

C	  loopa

D	        pupa

E	  loopa

F	  loopa



#3

Продолжим исследование героев Dota2. Сгруппируйте по колонкам attack_type и primary_attr и выберите самый распространённый набор характеристик.

In [127]:
#df = pd.read_csv('https://stepik.org/media/attachments/course/4852/dota_hero_stats.csv')
df = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/Stepik_Karpov_ML/main/dota_hero_stats.csv')
df.head()

,Unnamed: 0,attack_type,id,legs,localized_name,name,primary_attr,roles
0,0,Melee,1,2,Anti-Mage,npc_dota_hero_antimage,agi,"['Carry', 'Escape', 'Nuker']"
1,1,Melee,2,2,Axe,npc_dota_hero_axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']"
2,2,Ranged,3,4,Bane,npc_dota_hero_bane,int,"['Support', 'Disabler', 'Nuker', 'Durable']"
3,3,Melee,4,2,Bloodseeker,npc_dota_hero_bloodseeker,agi,"['Carry', 'Disabler', 'Jungler', 'Nuker', 'Ini..."
4,4,Ranged,5,2,Crystal Maiden,npc_dota_hero_crystal_maiden,int,"['Support', 'Disabler', 'Nuker', 'Jungler']"


In [ ]:
# df.describe()

In [128]:
df.groupby(['attack_type', 'primary_attr']).agg('count')

Unnamed: 0  id  legs  localized_name  name  roles
attack_type primary_attr                                                   
Melee       agi                   19  19    19              19    19     19
            int                    2   2     2               2     2      2
            str                   35  35    35              35    35     35
Ranged      agi                   18  18    18              18    18     18
            int                   40  40    40              40    40     40
            str                    3   3     3               3     3      3

In [129]:
df.groupby(['attack_type', 'primary_attr']).agg({'id':'count'}).rename(columns={'id': 'count'}).sort_values('count', ascending=False)

count
attack_type primary_attr       
Ranged      int              40
Melee       str              35
            agi              19
Ranged      agi              18
            str               3
Melee       int               2

In [130]:
df.groupby(['attack_type', 'primary_attr']).size()

attack_type  primary_attr
Melee        agi             19
             int              2
             str             35
Ranged       agi             18
             int             40
             str              3
dtype: int64

In [131]:
df.filter(items=['attack_type','primary_attr']).mode()

,attack_type,primary_attr
0,Ranged,int


Выберите один вариант из списка


Ranged int

#4

Аспирант Ростислав изучает метаболом водорослей и получил такую табличку. В ней он записал вид каждой водоросли, её род (группа, объединяющая близкие виды), группа (ещё одно объединение водорослей в крупные фракции) и концентрации анализируемых веществ.

Помогите Ростиславу найти среднюю концентрацию каждого из веществ в каждом из родов (колонка genus)! Для этого проведите группировку датафрэйма, сохранённого в переменной concentrations, и примените метод, сохранив результат в переменной mean_concentrations.

In [132]:
#concentrations = pd.read_csv('http://stepik.org/media/attachments/course/4852/algae.csv')
concentrations = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/Stepik_Karpov_ML/main/algae.csv')
concentrations.head()

,species,genus,group,sucrose,alanin,citrate,glucose,oleic_acid
0,Fucus_vesiculosus,Fucus,brown,3.001472,3.711498,5.004262,2.548459,6.405165
1,Saccharina_japonica,Saccharina,brown,6.731070,1.255251,5.621499,6.013219,4.156700
2,Fucus_serratus,Fucus,brown,3.276870,0.346431,1.216767,3.623225,0.304573
3,Fucus_distichus,Fucus,brown,6.786996,6.641303,6.423606,2.272724,3.393203
4,Cladophora_fracta,Cladophora,green,3.861470,1.648450,6.940588,2.316955,2.528886


In [133]:
mean_concentrations = concentrations.groupby('genus').agg({'sucrose': 'mean', 'alanin': 'mean', 'citrate': 'mean',\
                                                            'glucose': 'mean', 'oleic_acid': 'mean'})
mean_concentrations

,sucrose,alanin,citrate,glucose,oleic_acid
genus,,,,,
Ascophyllum,6.825467,0.875429,5.253527,3.414961,2.432526
Cladophora,4.008792,3.997055,5.288311,2.800276,2.263472
Fucus,4.355112,3.566411,4.214878,2.814803,3.367647
Palmaria,0.704580,3.176440,5.573905,3.242090,2.245538
Saccharina,4.183596,3.524207,3.346710,4.980594,4.487252


In [134]:
mean_concentrations = concentrations.groupby('genus').mean()
mean_concentrations

,sucrose,alanin,citrate,glucose,oleic_acid
genus,,,,,
Ascophyllum,6.825467,0.875429,5.253527,3.414961,2.432526
Cladophora,4.008792,3.997055,5.288311,2.800276,2.263472
Fucus,4.355112,3.566411,4.214878,2.814803,3.367647
Palmaria,0.704580,3.176440,5.573905,3.242090,2.245538
Saccharina,4.183596,3.524207,3.346710,4.980594,4.487252


In [135]:
mean_concentrations = concentrations.groupby('genus').agg(['mean'])
mean_concentrations

,sucrose,alanin,citrate,glucose,oleic_acid
,mean,mean,mean,mean,mean
genus,,,,,
Ascophyllum,6.825467,0.875429,5.253527,3.414961,2.432526
Cladophora,4.008792,3.997055,5.288311,2.800276,2.263472
Fucus,4.355112,3.566411,4.214878,2.814803,3.367647
Palmaria,0.704580,3.176440,5.573905,3.242090,2.245538
Saccharina,4.183596,3.524207,3.346710,4.980594,4.487252


#5

Пользуясь предыдущими данными, укажите через пробел (без запятых) чему равны минимальная, средняя и максимальная концентрации аланина (alanin) среди видов рода Fucus. Округлите до 2-ого знака, десятичным разделителем является точка.

Формат ответа:
0.55 6.77 7.48

In [136]:
concentrations.loc[concentrations['genus'] == 'Fucus'].describe()

,sucrose,alanin,citrate,glucose,oleic_acid
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,4.355112,3.566411,4.214878,2.814803,3.367647
std,2.110570,3.149943,2.691679,0.713559,3.050376
min,3.001472,0.346431,1.216767,2.272724,0.304573
25%,3.139171,2.028965,3.110515,2.410591,1.848888
50%,3.276870,3.711498,5.004262,2.548459,3.393203
75%,5.031933,5.176401,5.713934,3.085842,4.899184
max,6.786996,6.641303,6.423606,3.623225,6.405165


In [137]:
concentrations.groupby('genus').agg(['min', 'mean', 'max'])

sucrose                        alanin                      \
                  min      mean       max       min      mean       max   
genus                                                                     
Ascophyllum  6.825467  6.825467  6.825467  0.875429  0.875429  0.875429   
Cladophora   2.452623  4.008792  5.712284  1.648450  3.997055  6.881024   
Fucus        3.001472  4.355112  6.786996  0.346431  3.566411  6.641303   
Palmaria     0.704580  0.704580  0.704580  3.176440  3.176440  3.176440   
Saccharina   1.636122  4.183596  6.731070  1.255251  3.524207  5.793163   

              citrate                       glucose                      \
                  min      mean       max       min      mean       max   
genus                                                                     
Ascophyllum  5.253527  5.253527  5.253527  3.414961  3.414961  3.414961   
Cladophora   3.082826  5.288311  6.940588  2.316955  2.800276  3.343707   
Fucus        1.216767  4.214878  6.423606  2.272724  2.814803  3.623225   
Palmaria     5.573905  5.573905  5.573905  3.242090  3.242090  3.242090   
Saccharina   1.071920  3.346710  5.621499  3.947968  4.980594  6.013219   

            oleic_acid                      
                   min      mean       max  
genus                                       
Ascophyllum   2.432526  2.432526  2.432526  
Cladophora    1.432514  2.263472  2.829016  
Fucus         0.304573  3.367647  6.405165  
Palmaria      2.245538  2.245538  2.245538  
Saccharina    4.156700  4.487252  4.817804

In [138]:
concentrations.groupby('genus').agg(['min', 'mean', 'max']).loc['Fucus', 'alanin'].round(2)

min     0.35
mean    3.57
max     6.64
Name: Fucus, dtype: float64

In [139]:
np.around(concentrations.query("genus == 'Fucus'").alanin.describe().loc[['min', 'mean', 'max']].values, decimals=2)

array([0.35, 3.57, 6.64])

In [140]:
concentrations.query('genus == "Fucus"').describe()

,sucrose,alanin,citrate,glucose,oleic_acid
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,4.355112,3.566411,4.214878,2.814803,3.367647
std,2.110570,3.149943,2.691679,0.713559,3.050376
min,3.001472,0.346431,1.216767,2.272724,0.304573
25%,3.139171,2.028965,3.110515,2.410591,1.848888
50%,3.276870,3.711498,5.004262,2.548459,3.393203
75%,5.031933,5.176401,5.713934,3.085842,4.899184
max,6.786996,6.641303,6.423606,3.623225,6.405165


In [142]:
concentrations.query("genus == 'Fucus'")

,species,genus,group,sucrose,alanin,citrate,glucose,oleic_acid
0,Fucus_vesiculosus,Fucus,brown,3.001472,3.711498,5.004262,2.548459,6.405165
2,Fucus_serratus,Fucus,brown,3.276870,0.346431,1.216767,3.623225,0.304573
3,Fucus_distichus,Fucus,brown,6.786996,6.641303,6.423606,2.272724,3.393203


In [141]:
concentrations.query("genus == 'Fucus'").aggregate({'alanin': ['max','mean','min'] })

,alanin
max,6.641303
mean,3.566411
min,0.346431


нашли всех fucus

In [143]:
concentrations.loc[concentrations['genus'] == 'Fucus']

,species,genus,group,sucrose,alanin,citrate,glucose,oleic_acid
0,Fucus_vesiculosus,Fucus,brown,3.001472,3.711498,5.004262,2.548459,6.405165
2,Fucus_serratus,Fucus,brown,3.276870,0.346431,1.216767,3.623225,0.304573
3,Fucus_distichus,Fucus,brown,6.786996,6.641303,6.423606,2.272724,3.393203


аггрегируем по аланину

In [144]:
concentrations.loc[concentrations['genus'] == 'Fucus'].agg({'alanin': [np.min, np.mean, np.max ]}).round(2)

,alanin
amin,0.35
mean,3.57
amax,6.64


In [145]:
concentrations.aggregate({'alanin':['min','mean','max']})

,alanin
min,0.346431
mean,3.379068
max,6.881024


#6

Сгруппируйте данные(aglae) по переменной group и соотнесите вопросы с ответами

In [146]:
df = pd.read_csv('https://raw.githubusercontent.com/nedokormysh/Stepik_Karpov_ML/main/algae.csv')
df.head()

,species,genus,group,sucrose,alanin,citrate,glucose,oleic_acid
0,Fucus_vesiculosus,Fucus,brown,3.001472,3.711498,5.004262,2.548459,6.405165
1,Saccharina_japonica,Saccharina,brown,6.731070,1.255251,5.621499,6.013219,4.156700
2,Fucus_serratus,Fucus,brown,3.276870,0.346431,1.216767,3.623225,0.304573
3,Fucus_distichus,Fucus,brown,6.786996,6.641303,6.423606,2.272724,3.393203
4,Cladophora_fracta,Cladophora,green,3.861470,1.648450,6.940588,2.316955,2.528886


In [ ]:
df.groupby('group').describe()

sucrose                      ... oleic_acid                    
        count      mean       std  ...        50%       75%       max
group                              ...                               
brown     6.0  4.709666  2.336471  ...   3.774952  4.652528  6.405165
green     3.0  4.008792  1.634817  ...   2.528886  2.678951  2.829016
red       1.0  0.704580       NaN  ...   2.245538  2.245538  2.245538

[3 rows x 40 columns]

In [ ]:
# concentrations.groupby('group').agg({'citrate': 'max'})

In [147]:
df.groupby('group') \
    .agg({'citrate': 'var', 'sucrose': lambda x: x.max() - x.min(), 'glucose': 'count'}) \
    .round(2)

,citrate,sucrose,glucose
group,,,
brown,5.47,5.19,6
green,3.95,3.26,3
red,NaN,0.00,1


дисперсия концентрации цитрата в бурых, зелёных и красных водорослях
5.47, 3.95, NaN

Размах (максимум - минимум) сахарозы в бурых, зелёных и красных водорослях
5.19, 3.26, 0

Число видов бурых, зелёных и красных водорослей
6, 3, 1

In [148]:
df.groupby('group').aggregate(['count', 'var', np.ptp])

sucrose                     alanin                     citrate  \
        count       var       ptp  count       var       ptp   count   
group                                                                  
brown       6  5.459098  5.189345      6  7.220351  6.294872       6   
green       3  2.672626  3.259662      3  7.059917  5.232574       3   
red         1       NaN  0.000000      1       NaN  0.000000       1   

                          glucose                     oleic_acid            \
            var       ptp   count       var       ptp      count       var   
group                                                                        
brown  5.468942  5.351686       6  1.767756  3.740495          6  4.385233   
green  3.950113  3.857763       3  0.266265  1.026753          3  0.540388   
red         NaN  0.000000       1       NaN  0.000000          1       NaN   

                 
            ptp  
group            
brown  6.100592  
green  1.396502  
red    0.000000